Task 1: Third-order letter approximation model

In [1]:
with open('/workspaces/emergingtechnologies/Data/Frankenstien.txt', 'r') as f:
    text = f.read()

In [2]:
#define the preamble and postamble markers
start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK FRANKENSTEIN; OR, THE MODERN PROMETHEUS ***"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK FRANKENSTEIN; OR, THE MODERN PROMETHEUS ***"

#find the start and end markers in the text
start = text.find(start_marker)
end = text.find(end_marker)

#ensure both markers are found
if start != -1 and end != -1:
    #adjust the start index to skip the start marker
    start += len(start_marker)
    #extract and clean the text between the markers
    text = text[start:end].strip()



In [3]:
#clean the text by removing all characters except for uppercase letters, spaces, and periods
chars = set('ABCDEFGHIJKLMNOPQRSTUVWXYZ. ')
#convert the text to uppercase
text = text.upper()
#remove all characters that are not in the set of allowed characters
cleaned = ''.join(c for c in text if c in chars)


In [4]:
model = {}

#iterate over the text and count the occurrences of each trigram
for i in range(len(cleaned) - 1):
  digram = cleaned[i:i+3]
  model[digram] = model.get(digram, 0) + 1

In [5]:
sorted(model.items(), key=lambda x: x[1], reverse=True)

[(' TH', 6698),
 ('THE', 5663),
 ('HE ', 4576),
 ('ED ', 4117),
 ('ND ', 3393),
 ('AND', 3286),
 (' AN', 3260),
 (' OF', 2599),
 ('OF ', 2436),
 (' I ', 2422),
 (' TO', 2299),
 ('TO ', 2063),
 (' MY', 1818),
 (' IN', 1784),
 ('ER ', 1769),
 ('D T', 1722),
 ('ING', 1722),
 ('MY ', 1607),
 ('AS ', 1538),
 ('AT ', 1528),
 (' WH', 1460),
 ('NG ', 1448),
 ('E A', 1444),
 ('ON ', 1408),
 (' HA', 1405),
 ('RE ', 1405),
 ('IN ', 1352),
 ('E T', 1324),
 (' BE', 1320),
 (' HE', 1301),
 (' CO', 1277),
 ('ME ', 1274),
 (' A ', 1202),
 ('HAT', 1201),
 ('THA', 1194),
 (' WA', 1194),
 ('HER', 1174),
 ('D A', 1170),
 ('ION', 1156),
 ('IS ', 1137),
 ('ENT', 1131),
 (' WI', 1077),
 (' RE', 1070),
 ('EN ', 1052),
 (' ME', 1051),
 ('FOR', 1045),
 ('ERE', 1034),
 ('WAS', 1030),
 ('E S', 1024),
 ('E W', 1023),
 ('N T', 980),
 ('HIS', 980),
 ('TH ', 971),
 ('LY ', 965),
 ('S A', 962),
 ('ES ', 942),
 (' DE', 935),
 (' NO', 935),
 ('YOU', 928),
 ('E O', 903),
 ('TIO', 897),
 ('VER', 890),
 ('D I', 889),
 ('T 